In [169]:
from openai import OpenAI
import pandas as pd
from dataProcessor import process_metadata, pew_metadata_path, statista_metadata_path

In [170]:
#  get the combined dataframe
combined_df = process_metadata(pew_metadata_path, statista_metadata_path)


In [181]:
# Extract the 'title' and 'caption' columns
data = combined_df[['title', 'caption']].head(12061)
# Display the new DataFrame
data

,title,caption
0,"Foreign-born population in the United States, ...",The foreign-born population residing in the U....
1,"English proficiency among U.S. immigrants, 198...","Since 1980, the share of immigrants who are pr..."
2,"Languages spoken among U.S. immigrants, 2018","Among the nation’s immigrants, Spanish is by f..."
3,"Hispanic population in the U.S., 2000-2017",There were nearly 60 million Latinos in the Un...
4,Weekly broadcast audience for top 20 NPR-affil...,The top 20 NPR-affiliated public radio station...
...,...,...
12056,\r\n Average age of the...,This statistic shows the average age of the f...
12057,\r\n Number of direct o...,"In 2019, Todâ€™s Group, which controls the br..."
12058,\r\n Breakdown of compu...,The statistic above presents a breakdown of U...
12059,\r\n Most common action...,"In 2019, 45 percent of global data breaches f..."


In [177]:
def context(title, caption):
    return f"Title: '{title}' \n Caption: '{caption}'"

# Example usage
content = data.apply(lambda row: context(row['title'], row['caption']), axis=1)

In [178]:
content

0        Title: 'Foreign-born population in the United ...
1        Title: 'English proficiency among U.S. immigra...
2        Title: 'Languages spoken among U.S. immigrants...
3        Title: 'Hispanic population in the U.S., 2000-...
4        Title: 'Weekly broadcast audience for top 20 N...
                               ...                        
12055    Title: '\r\n                        Most impor...
12056    Title: '\r\n                        Average ag...
12057    Title: '\r\n                        Number of ...
12058    Title: '\r\n                        Breakdown ...
12059    Title: '\r\n                        Most commo...
Length: 12060, dtype: object

In [163]:
# Create an OpenAI client with your deepinfra token and endpoint
openai = OpenAI(
    api_key="xazaQpv4cJkR08ECanjwMStECgdRdd4t",
    base_url="https://api.deepinfra.com/v1/openai",
)

In [164]:
def generate_question(content):
    response = openai.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-70B-Instruct",
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an AI assistant. Generate a 1 line question based on the following information.\n\n"
                    "Here are some example questions to illustrate the desired question format:\n"
                    "Are gas prices too high?\n"
                    "Is a college education worth it?\n"
                    "Is capitalism the best form of economy?\n"
                    "Do we need cash?\n"
                    "Should education be free?\n"
                    "Does lowering the federal corporate income tax rate create jobs?\n"
                    "Should everyone get a universal basic income?\n"
                    "Should the press be subsidized?\n"
                    "Does legal prostitution increase human trafficking?\n"
                    "Should the penny stay in circulation?\n"
                    "Should government spending be reduced?\n"
                    "Should blood donations be financially compensated?\n"
                    "Should prescription drugs be advertised directly to consumers?\n"
                    "Does poverty cause crime?"
                )    
             },
            {"role": "user", "content": content}
        ],
        max_tokens=20,  # Limit the response to ensure it's a single line
        temperature=0.5
    )
    question = (response.choices[0].message.content)
    return question

In [165]:
def classify_sector(question):
    response = openai.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-70B-Instruct",
        messages=[
            {"role": "system", "content": "You are an AI assistant. Classify the following question into a relevant sector in 1 word."},
            {"role": "user", "content": question}
        ],
        max_tokens=5,  # A short response is expected for the sector name
    )
    sector = (response.choices[0].message.content)
    return sector

In [166]:
from tqdm import tqdm
# Initialize lists to store the generated questions and sectors
questions = []
sectors = []

# Loop through each row in the DataFrame
for index, row in tqdm(data.iterrows(),total=data.shape[0], desc="Processing Rows"):
    content = context(row['title'], row['caption'])  # Using the context function to create the prompt
    # Generate the question
    question = generate_question(content)
    questions.append(question)
    
    # Classify the sector based on the generated question
    sector = classify_sector(question)
    sectors.append(sector)

Processing Rows:  41%|████      | 12060/29354 [8:24:55<12:04:03,  2.51s/it] 


InternalServerError: Error code: 500 - {'error': {'message': 'inference error'}}

In [182]:
question_df = data[['title']]

In [186]:
sectors.append('Media')

In [187]:
# Adding the questions and sectors to the DataFrame
question_df['Question'] = questions
question_df['Sector'] = sectors
question_df

,title,Question,Sector
0,"Foreign-born population in the United States, ...",Should the United States increase its annual l...,Politics.
1,"English proficiency among U.S. immigrants, 198...",Should immigrants to the U.S. be required to d...,Immigration
2,"Languages spoken among U.S. immigrants, 2018",Should the US government provide more support ...,Immigration
3,"Hispanic population in the U.S., 2000-2017",Should the growing Hispanic population in the ...,Politics
4,Weekly broadcast audience for top 20 NPR-affil...,Should NPR receive continued government funding?,Politics
...,...,...,...
12056,\r\n Average age of the...,Is the average age of first-time fathers incre...,Demographics.
12057,\r\n Number of direct o...,Should luxury fashion brands like Tod's priori...,Retail
12058,\r\n Breakdown of compu...,Should video games rated for teenagers be subj...,Entertainment
12059,\r\n Most common action...,Are data breaches more often caused by hacking...,Cybersecurity


In [188]:
# Save the DataFrame to a CSV file
question_df.to_csv("classified_questions.csv", index=False)

In [5]:
import pandas as pd
#read the csv file
question_df = pd.read_csv('../dataset/classified_questions.csv')

In [6]:
question_df

,title,Question,Sector
0,"Foreign-born population in the United States, ...",Should the United States increase its annual l...,Politics.
1,"English proficiency among U.S. immigrants, 198...",Should immigrants to the U.S. be required to d...,Immigration
2,"Languages spoken among U.S. immigrants, 2018",Should the US government provide more support ...,Immigration
3,"Hispanic population in the U.S., 2000-2017",Should the growing Hispanic population in the ...,Politics
4,Weekly broadcast audience for top 20 NPR-affil...,Should NPR receive continued government funding?,Politics
...,...,...,...
12056,\r\n Average age of the...,Is the average age of first-time fathers incre...,Demographics.
12057,\r\n Number of direct o...,Should luxury fashion brands like Tod's priori...,Retail
12058,\r\n Breakdown of compu...,Should video games rated for teenagers be subj...,Entertainment
12059,\r\n Most common action...,Are data breaches more often caused by hacking...,Cybersecurity


In [8]:
from dataProcessor import process_metadata, clean_text, correct_misinterpreted_characters, pew_metadata_path, statista_metadata_path, llava_description_path

# Apply text cleaning and correction functions
question_df['title'] = question_df['title'].apply(clean_text).apply(correct_misinterpreted_characters)
question_df['Question'] = question_df['Question'].apply(clean_text).apply(correct_misinterpreted_characters)

question_df


,title,Question,Sector
0,"Foreign-born population in the United States, ...",Should the United States increase its annual l...,Politics.
1,"English proficiency among U.S. immigrants, 198...",Should immigrants to the U.S. be required to d...,Immigration
2,"Languages spoken among U.S. immigrants, 2018",Should the US government provide more support ...,Immigration
3,"Hispanic population in the U.S., 2000-2017",Should the growing Hispanic population in the ...,Politics
4,Weekly broadcast audience for top 20 NPR-affil...,Should NPR receive continued government funding?,Politics
...,...,...,...
12056,Average age of the father at the birth of the ...,Is the average age of first-time fathers incre...,Demographics.
12057,Number of direct operating stores (DOS) of the...,Should luxury fashion brands like Tod's priori...,Retail
12058,Breakdown of computer and video games in the U...,Should video games rated for teenagers be subj...,Entertainment
12059,Most common action varieties in data breaches ...,Are data breaches more often caused by hacking...,Cybersecurity


In [9]:
# Save the DataFrame to a CSV file
question_df.to_csv("../dataset/classified_questions.csv'", index=False)